#### About
> Multi task learning

It is a powerful technique in natural language processing that allows us to train a single model to perform multiple tasks.

In [1]:
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, pipeline
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


/home/suraj/anaconda3/envs/dl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-13 23:06:03.415276: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-13 23:06:13.807405: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# Load synthetic dataset
df = pd.read_csv('train.csv')


In [ ]:
# Convert language column to categorical variable
df['language'] = pd.Categorical(df['language'])
num_classes_language = len(df['language'].cat.categories)


In [ ]:
# Convert sentiment column to numerical labels
sentiment_labels = {'1 star': 0, '2 stars': 1, '3 stars': 2, '4 stars': 3, '5 stars': 4}
df['sentiment'] = df['sentiment'].apply(lambda x: sentiment_labels[x])
num_classes_sentiment = len(sentiment_labels)


In [ ]:
# Load GPT-2 tokenizer
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [ ]:
# Tokenize tweets
encoded = tokenizer(df['tweet'].tolist(), padding=True, truncation=True, max_length=30, return_tensors='tf')



In [ ]:
# Split data into train and test sets
train_inputs, test_inputs, train_labels, test_labels = train_test_split(encoded['input_ids'], 
                                                                        {'language': to_categorical(df['language'].cat.codes, num_classes=num_classes_language),
                                                                         'sentiment': to_categorical(df['sentiment'], num_classes=num_classes_sentiment)},
                                                                        test_size=0.2, random_state=42)


In [ ]:
languages = len(df['language'].cat.codes)

In [ ]:
# Load the pre-trained sentiment analysis model
sentiment_model = TFAutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Define the input layer
input_layer = Input(shape=(train_inputs.shape[1],), dtype='int32')

# Add the pre-trained sentiment analysis model on top of the input layer
outputs = sentiment_model(input_layer)[0]

# Add a dense layer on top of the sentiment analysis model to predict language
language_predictions = Dense(units=len(languages), activation='softmax', name='language')(outputs)

# Add a dense layer on top of the sentiment analysis model to predict sentiment
sentiment_predictions = Dense(units=5, activation='softmax', name='sentiment')(outputs)

# Create the multi-task learning model
model = Model(inputs=input_layer, outputs=[language_predictions, sentiment_predictions])

# Compile the model
model.compile(optimizer=Adam(learning_rate=5e-5), loss={'language': 'categorical_crossentropy', 'sentiment': 'categorical_crossentropy'}, metrics={'language': 'accuracy', 'sentiment': 'accuracy'})



In [ ]:
# Train the model
history = model.fit(train_inputs, {'language': np.eye(len(languages))[train_labels[:,0]], 'sentiment': np.eye(5)[train_labels[:,1]-1]}, epochs=10, batch_size=32, validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)])



In [ ]:
# Evaluate the model on the test set
test_loss, test_acc_language, test_acc_sentiment = model.evaluate(test_inputs, {'language': np.eye(len(languages))[test_labels[:,0]], 'sentiment': np.eye(5)[test_labels[:,1]-1]})

print('Test accuracy for language prediction:', test_acc_language)
print('Test accuracy for sentiment prediction:', test_acc_sentiment)